In [9]:
# from teamName.py
#!/usr/bin/envblah python

import numpy as np
import pandas as pd
# from teamName import getMyPosition as getPosition

# Algorithm testing file. 
# Quantitative judging will be determined from output of this program.
# Judging will use unseeen, future price data from the same universe.

nInst = 0
nt = 0

# Commission rate.
commRate = 0.0050

# Dollar position limit (maximum absolute dollar value of any individual stock position).
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    nt, nInst = df.values.shape
    return (df.values).T

pricesFile="./prices250.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolume0 = 0
    totDVolume1 = 0
    frac0 = 0.
    frac1 = 0.
    value = 0
    todayPLL = []
    (_,nt) = prcHist.shape
    for t in range(201,251):
        prcHistSoFar = prcHist[:,:t]
        newPosOrig = getPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:,-1] 
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.array([int(p) for p in np.clip(newPosOrig, -posLimits, posLimits)])
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume0 = np.sum(dvolumes[:50])
        dvolume1 = np.sum(dvolumes[50:])
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        totDVolume0 += dvolume0
        totDVolume1 += dvolume1
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
            frac0 = totDVolume0 / totDVolume
            frac1 = totDVolume1 / totDVolume
        # old print
        # print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf frac0: %.4lf frac1: %.4lf" % (t,value, todayPL, totDVolume, ret, frac0, frac1))
        print ("Day %d value: %9.2lf todayPL: $%8.2lf $-traded: %7.0lf return: %8.5lf frac0: %6.4lf frac1: %6.4lf" % (t,value, todayPL, totDVolume, ret, frac0, frac1))
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = 16 * plmu / plstd
    return (plmu, ret, annSharpe, totDVolume)

Loaded 100 instruments for 250 days


# Define strategy here

In [52]:
# buy and hold

# nInst = 100   # number of instruments
# nt = 250      # number of days

currentPos = np.zeros(nInst)
currentPos[0] = 1000000 # the position limit will make it so we buy max position
day = 201

def getMyPosition(prcSoFar):
    global day
    global currentPos
    if day == 250:  # selling at end of day does affect results
        currentPos[0] = 0
    day += 1
    print(day, currentPos[0])
    return currentPos

In [58]:
def transactionCost(currentPrices, currentPos, newPos):
    """
    Returns transaction cost for new position.

        transaction cost = 0.0050 * totalDollarVolumeTraded

    from eval.py
    """
    # Commission rate
    commRate = 0.0050
    deltaPos = newPos - currentPos
    dvolumes = currentPrices * np.abs(deltaPos)
    dvolume0 = np.sum(dvolumes[:50])
    dvolume1 = np.sum(dvolumes[50:])
    dvolume = np.sum(dvolumes)
    return dvolume * commRate

In [65]:
# test transaction cost
prices = np.loadtxt("prices250.txt")
buyOne = np.zeros(nInst)
buyOne[0] = 500
print(transactionCost(prices[0], np.zeros(nInst), buyOne))
print("Check:", prices[0][0] * 500 * 0.0050)

45.625
Check: 45.625


In [8]:
nInst=100
currentPos = np.zeros(nInst)

# Dummy algorithm to demonstrate function format.
def getMyPosition (prcSoFar):
    global currentPos
    # (nins,nt) = prcSoFar.shape
    # rpos = np.array([int(x) for x in 1000 * np.random.randn(nins)])
    # currentPos += rpos
    # The algorithm must return a vector of integers, indicating the position of each stock.
    # Position = number of shares, and can be positve or negative depending on long/short position.
    return currentPos


In [53]:
# Run calcPL here from eval.py

getPosition = getMyPosition


# Output
(meanpl, ret, sharpe, dvol) = calcPL(prcAll)
print ("=====")
print ("mean(PL): %.0lf" % meanpl)
print ("return: %.5lf" % ret)
print ("annSharpe(PL): %.2lf " % sharpe)
print ("totDvolume: %.0lf " % dvol)

202 1000000.0
Day 201 value:    -49.99 todayPL: $  -49.99 $-traded:    9998 return: -0.00500 frac0: 1.0000 frac1: 0.0000
203 1000000.0
Day 202 value:     60.95 todayPL: $  110.94 $-traded:   10118 return:  0.00602 frac0: 1.0000 frac1: 0.0000
204 1000000.0
Day 203 value:      0.53 todayPL: $  -60.42 $-traded:   10177 return:  0.00005 frac0: 1.0000 frac1: 0.0000
205 1000000.0
Day 204 value:    -65.28 todayPL: $  -65.82 $-traded:   10236 return: -0.00638 frac0: 1.0000 frac1: 0.0000
206 1000000.0
Day 205 value:   -131.58 todayPL: $  -66.30 $-traded:   10314 return: -0.01276 frac0: 1.0000 frac1: 0.0000
207 1000000.0
Day 206 value:   -131.58 todayPL: $    0.00 $-traded:   10314 return: -0.01276 frac0: 1.0000 frac1: 0.0000
208 1000000.0
Day 207 value:    -24.87 todayPL: $  106.72 $-traded:   10433 return: -0.00238 frac0: 1.0000 frac1: 0.0000
209 1000000.0
Day 208 value:    -75.66 todayPL: $  -50.80 $-traded:   10492 return: -0.00721 frac0: 1.0000 frac1: 0.0000
210 1000000.0
Day 209 value:    